In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

import plotly.express as px

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'silking_progress', 'w_df_all', 'w_w_df_all'])

In [28]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)

model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [36]:
date_start=dt(2022,4,1)
date_end=dt(2022,7,7)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF = cy.Build_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
pred_DF

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-04-01,1.0,2022,NaN,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,58.816266,64.895429
2022-04-02,1.0,2022,NaN,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,58.816266,64.895429
2022-04-03,1.0,2022,NaN,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,58.816266,64.895429
2022-04-04,1.0,2022,NaN,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,58.816266,64.895429
2022-04-05,1.0,2022,NaN,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,58.816266,64.895429
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-03,1.0,2022,NaN,49.000000,8.351362,69.745245,9.162929,83.959270,76.522937,58.816266,81.560978
2022-07-04,1.0,2022,NaN,49.000000,8.351362,69.745245,9.075696,82.368264,75.794424,58.816266,81.311503
2022-07-05,1.0,2022,NaN,49.000000,8.351362,69.745245,9.055826,82.007982,75.628479,58.434579,81.311503
2022-07-06,1.0,2022,NaN,49.000000,8.351362,69.745245,9.043891,81.791970,75.528809,57.848393,81.311503


In [39]:
date_start=dt(2022,4,1)
date_end=dt(2022,7,7)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
pred_DF
# yields=model.predict(pred_DF[model.params.index])
# pred_DF['Yield']=yields
# print('Base Yield:',yields.values[-1])

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-04-01,1.0,2022,NaN,72.289575,0.022150,0.000491,0.022150,0.000491,0.000491,0.000000,0.000000
2022-04-02,1.0,2022,NaN,72.289575,0.066310,0.004397,0.066310,0.004397,0.004397,0.000000,0.000000
2022-04-03,1.0,2022,NaN,72.289575,0.039742,0.001579,0.039742,0.001579,0.001579,0.000000,0.000000
2022-04-04,1.0,2022,NaN,72.289575,0.066066,0.004365,0.066066,0.004365,0.004365,0.000000,0.000000
2022-04-05,1.0,2022,NaN,72.289575,0.033136,0.001098,0.033136,0.001098,0.001098,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-03,1.0,2022,NaN,49.000000,8.351362,69.745245,1.135778,1.289992,9.485293,1.362540,33.379188
2022-07-04,1.0,2022,NaN,49.000000,8.351362,69.745245,1.173051,1.376048,9.796571,1.556117,34.741728
2022-07-05,1.0,2022,NaN,49.000000,8.351362,69.745245,1.292318,1.670085,10.792613,1.678643,36.297845
2022-07-06,1.0,2022,NaN,49.000000,8.351362,69.745245,1.400996,1.962790,11.700225,3.215855,36.297845


In [5]:
ext_seas = uw.extend_with_seasonal_df(raw_data['w_w_df_all'][GV.WD_H_GFS])
ext_seas

,USA_Prec,USA_TempMax,USA_Sdd30
time,,,
1985-01-01,11.863999,-1.383573,0.0
1985-01-02,0.568994,-3.520978,0.0
1985-01-03,0.047351,-4.898498,0.0
1985-01-04,0.973402,-1.346037,0.0
1985-01-05,0.224217,2.471312,0.0
...,...,...,...
2022-12-27,1.704770,1.221858,0.0
2022-12-28,1.734577,1.342461,0.0
2022-12-29,1.964567,1.505311,0.0


In [7]:
ext_seas['2022-04-16':'2022-06-20']['USA_Prec'].sum()

212.12459110014947

In [15]:
ext_seas_cum_sum=ext_seas.cumsum()
ext_seas_cum_sum.loc['2022-06-20']-ext_seas_cum_sum.loc['2022-04-15']

USA_Prec        212.124591
USA_TempMax    1460.716711
USA_Sdd30        23.128946
dtype: float64

In [16]:
dt_1= dt(2022,4,1)
dt_2= dt(2022,8,1)

In [19]:
dt_3 = min(dt_1,dt_2)
dt_3

datetime.datetime(2022, 4, 1, 0, 0)

In [ ]:

ext_seas.loc['2022-07-04':'2022-08-03','USA_Sdd30'].sum()

In [ ]:
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF = cy.Build_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(pred_DF[model.params.index])
pred_DF['Yield']=yields
print('Base Yield:',yields.values[-1])

ext_dict={'Prec' :'Mean','Sdd30':'Analog_2012'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF_2012 = cy.Build_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(pred_DF_2012[model.params.index])
pred_DF_2012['Yield']=yields
print('2012 Yield:',yields.values[-1])

In [ ]:
px.line((pred_DF_2012-pred_DF).diff().cumsum()['Yield'])

In [ ]:
(pred_DF-pred_DF_2012).diff().cumsum()

In [ ]:
pred_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
base_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
d_pred_chart=pd.concat([d_pred_DF_2012['Yield'],d_pred_DF['Yield']],axis=1,keys=['2012','Base'])


fig=px.line(pred_DF['Yield']-pred_DF_2012['Yield']); fig.show()
fig=px.line(pred_chart); fig.show()
fig=px.line(d_pred_chart); fig.show()
# px.line(df_study)

# px.line(df_study)
